In [1]:
pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [6]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import dash
from dash import html, dcc, dash_table
from dash.dependencies import Input, Output
import numpy as np
df=pd.read_csv('/content/drive/MyDrive/final_data.csv')
df['DateTime'] = pd.to_datetime(df['DateTime'])



# Initialize Dash app
app = dash.Dash(__name__)


app.layout = html.Div([
    html.H1("Energy Consumption Analytics Dashboard",
            style={
                'textAlign': 'center',
                'color': '#1E90FF',
                'marginBottom': '30px'
            }),

    # Filters Container
    html.Div([
        # Filters Row
        html.Div([
            # Region Filter
            html.Div([
                html.Label("Region Filter", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id="region-filter",
                    options=[{"label": r, "value": r} for r in df['region'].unique()],
                    multi=True,
                    placeholder="Select Regions"
                )
            ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

            # Time of Day Filter
            html.Div([
                html.Label("Time of Day", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id="time-filter",
                    options=[{"label": t, "value": t} for t in df['TimeOfDay'].unique()],
                    multi=True,
                    placeholder="Select Time Periods"
                )
            ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

            # Consumption Category Filter
            html.Div([
                html.Label("Consumption Category", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id="category-filter",
                    options=[{"label": c, "value": c} for c in df['Consumption_Category'].unique()],
                    multi=True,
                    placeholder="Select Consumption Levels"
                )
            ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
        ]),

        # Date Range Picker
        html.Div([
            html.Label("Date Range", style={'fontWeight': 'bold'}),
            dcc.DatePickerRange(
                id="date-picker",
                start_date=df['DateTime'].min().date(),
                end_date=df['DateTime'].max().date(),
                display_format="YYYY-MM-DD"
            )
        ], style={'textAlign': 'center', 'marginTop': '20px'})
    ], style={
        'backgroundColor': '#F0F8FF',
        'padding': '20px',
        'borderRadius': '10px'
    }),

    # Visualization Grid
    html.Div([
        # First Row of Visualizations
        html.Div([
            dcc.Graph(id="bar-chart", style={'width': '50%', 'display': 'inline-block'}),
            dcc.Graph(id="line-chart", style={'width': '50%', 'display': 'inline-block'})
        ]),

        # Second Row of Visualizations
        html.Div([
            dcc.Graph(id="scatter-chart", style={'width': '50%', 'display': 'inline-block'}),
            dash_table.DataTable(
                id='data-table',
                columns=[{"name": i, "id": i} for i in ['region', 'TimeOfDay', 'New_KWH/hh_(per half hour)', 'cost']],
                style_table={'overflowX': 'auto'},
                page_size=10
            )
        ])
    ])
], style={'fontFamily': 'Arial, sans-serif'})

# Callback for updating visualizations
@app.callback(
    [
        Output("bar-chart", "figure"),
        Output("line-chart", "figure"),
        Output("scatter-chart", "figure"),
        Output("data-table", "data")
    ],
    [
        Input("region-filter", "value"),
        Input("time-filter", "value"),
        Input("category-filter", "value"),
        Input("date-picker", "start_date"),
        Input("date-picker", "end_date")
    ]
)
def update_visuals(selected_region, selected_time, selected_category, start_date, end_date):
    # Filter data with error handling
    filtered_df = df.copy()

    try:
        if selected_region:
            filtered_df = filtered_df[filtered_df['region'].isin(selected_region)]
        if selected_time:
            filtered_df = filtered_df[filtered_df['TimeOfDay'].isin(selected_time)]
        if selected_category:
            filtered_df = filtered_df[filtered_df['Consumption_Category'].isin(selected_category)]
        if start_date and end_date:
            filtered_df = filtered_df[
                (filtered_df['DateTime'].dt.date >= pd.to_datetime(start_date).date()) &
                (filtered_df['DateTime'].dt.date <= pd.to_datetime(end_date).date())
            ]
    except Exception as e:
        print(f"Error filtering data: {e}")

    # Bar Chart: Aggregated Energy Consumption by Region and Time of Day
    bar_chart = px.bar(
        filtered_df.groupby(['region', 'TimeOfDay'])['New_KWH/hh_(per half hour)'].mean().reset_index(),
        x="region",
        y="New_KWH/hh_(per half hour)",
        color="TimeOfDay",
        title="Average Energy Consumption by Region and Time",
        labels={"New_KWH/hh_(per half hour)": "Avg Energy Consumption (kWh)"}
    )

    # Line Chart: Energy Consumption Trends Over Time
    line_chart = px.line(
        filtered_df.groupby(['DateTime', 'region'])['New_KWH/hh_(per half hour)'].mean().reset_index(),
        x="DateTime",
        y="New_KWH/hh_(per half hour)",
        color="region",
        title="Energy Consumption Trends",
        labels={"New_KWH/hh_(per half hour)": "Energy Consumption (kWh)"}
    )

    # Scatter Plot: Cost vs Energy Consumption
    scatter_chart = px.scatter(
        filtered_df,
        x="New_KWH/hh_(per half hour)",
        y="cost",
        color="region",
        size="New_KWH/hh_(per half hour)",
        title="Energy Consumption vs Cost",
        labels={
            "New_KWH/hh_(per half hour)": "Energy Consumption (kWh)",
            "cost": "Total Cost"
        }
    )

    # Data Table
    table_data = filtered_df[['region', 'TimeOfDay', 'New_KWH/hh_(per half hour)', 'cost']].to_dict('records')

    return bar_chart, line_chart, scatter_chart, table_data

# Run server
if __name__ == "__main__":
    app.run_server(debug=True)

<IPython.core.display.Javascript object>